In [ ]:
import os
import shutil
import sys
import os.path as osp
sys.path
sys.path.append('./L1DeepMETv2/')
from graphmetnetwork import GraphMetNetwork

import numpy as np
import torch

from torch_cluster import radius_graph
from torch_geometric.data import DataLoader
import model.net as net
import model.data_loader as data_loader
import utils

### Load Parameters

In [ ]:
data_dir = './L1DeepMETv2/data_ttbar'
dataloaders = data_loader.fetch_dataloader(data_dir = data_dir, batch_size=1, validation_split=.2)
test_dl = dataloaders['test']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Test dataloader: {}'.format(len(test_dl)))
print(device)

### Load a sample test data point from the test dataloader

In [ ]:
test_data = None
for cnt, test_data in enumerate(test_dl):
    if cnt == 5:
        break

In [ ]:
test_data.x

In [ ]:
test_data.x.shape

#### Load Tensor Parameters

In [6]:
n_features_cont = 6
x_cont_test = test_data.x[:,:n_features_cont] .to(device)  # include puppi
x_cat_test = test_data.x[:,n_features_cont:].long().to(device)
etaphi_test = torch.cat([test_data.x[:, 3][:, None], test_data.x[:, 4][:, None]], dim=1).to(device=device)
batch_test = test_data.batch.to(device)
edge_index_test = radius_graph(etaphi_test, r=0.4, batch=batch_test, loop=False, max_num_neighbors=255).to(device=device)
print(f'x_cont_test: {x_cont_test.shape}')
print(f'x_cat_test: {x_cat_test.shape}')
print(f'etaphi: {etaphi_test.shape}')
print(f'batch: {batch_test.shape}')
print(f'edge_index: {edge_index_test.shape}')

#### Convert Tensor parameters to Numpy arrays

In [8]:
x_cont = np.ascontiguousarray(x_cont_test.squeeze(0).cpu().numpy())
x_cat = np.ascontiguousarray(x_cat_test.squeeze(0).cpu().numpy())
batch = np.ascontiguousarray(batch_test.squeeze(0).cpu().numpy())
etaphi = etaphi_test.squeeze(0).cpu().numpy()
edge_index = edge_index_test.squeeze(0).cpu().numpy().transpose()
num_nodes = x_cont.shape[0]
batch_size = batch.shape[0]
print(f'Number of nodes: {num_nodes}')
assert(num_nodes == batch_size)

### Load the Torch Model

In [ ]:
prefix = './L1DeepMETv2/ckpts_April30_scale_sigmoid'
restore_ckpt = osp.join(prefix, 'last.pth.tar')
norm = torch.tensor([1., 1., 1., 1., 1., 1.]).to(device=device)
torch_model = net.Net(continuous_dim=6, categorical_dim=2 , norm=norm).to(device)
torch_model.eval()
print(torch_model)

#### Get the weights

In [10]:
param_restored_new = utils.load_checkpoint(restore_ckpt, torch_model)
weights_dict = param_restored_new['state_dict']
print(weights_dict)

In [12]:
output_dir = "weights_files/"

# Check if the directory exists
if os.path.exists(output_dir):
    # Iterate over all the files in the directory
    for filename in os.listdir(output_dir):
        file_path = os.path.join(output_dir, filename)
        try:
            # Check if it's a file and delete it
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            # If it's a directory, delete the directory and its contents
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")
else:
    print(f"Directory {output_dir} does not exist.")


# Function to save the weights as binary files
def save_weights_as_binary(weights_dict, output_dir):
    for key, tensor in weights_dict.items():
        # Convert the tensor to a NumPy array
        np_array = tensor.cpu().numpy()

        # Create a binary file name based on the tensor name
        file_name = output_dir + key.replace('.', '_') + '.bin'

        # Save the NumPy array as a binary file
        np_array.tofile(file_name)
        
# Save all weights in the OrderedDict to binary files
save_weights_as_binary(weights_dict, output_dir)

### Load the C++ Model

In [12]:
output_dir = "weights_files/"

# Create an instance of the C++ GraphMetNetwork model
cmodel = GraphMetNetwork()

# Load the weights
cmodel.load_weights(output_dir)

### Test the weights

In [14]:
num_weights = 0
for key, tensor in weights_dict.items():
    # Convert the tensor to a NumPy array
    np_array = tensor.cpu().numpy()

    # Return cmodel function pointer to get the weight array
    cmodel_weight_func_name = 'get_' + key.replace('.', '_')
    cmodel_weight_func = getattr(cmodel, cmodel_weight_func_name)
    cmodel_weight_array = cmodel_weight_func()
    
    # Compare Torch model weight with Cmodel weight
    assert(np.allclose(np_array, cmodel_weight_array, atol=1e-5)), f'cmodel.{cmodel_weight_func_name} returned the wrong weights'
    num_weights += 1

print(f'Number of weights checked: {num_weights}')

### Run the Torch Model

In [13]:
results = torch_model(x_cont_test, x_cat_test, edge_index_test, batch_test)

### Run the C++ Model

In [14]:
cmodel.GraphMetNetworkLayers(x_cont, x_cat, batch, num_nodes)

#### Test Final Result

In [ ]:
# convert result from torch model to numpy arra
np_results = results.detach().cpu().numpy()
print(np_results)

In [30]:
np.testing.assert_allclose(np_results, cmodel.get_output(), rtol=1e-3)

In [31]:
c_results = cmodel.get_output()

In [32]:
print(c_results)